In [1]:
from os import chdir
chdir('../')
%matplotlib inline
from lib import *

pd.options.display.precision = 4
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Lato'
mpl.rcParams['font.weight'] = 700
sns.set(font='Lato', font_scale=1)
sns.set()
np.random.seed(42)
suppress_warnings()

In [2]:
feature = pd.read_pickle('/home/jovyan/work/DSI/projects/project_3/data/feature')
target= pd.read_pickle('/home/jovyan/work/DSI/projects/project_3/data/target')

In [3]:
display(feature.shape)
display(target.shape)

(10768, 1000)

(10768, 1)

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_classif
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso, LogisticRegression,LinearRegression
from sklearn.pipeline import Pipeline

# Step 2 Identify Features

In [5]:
from sklearn.feature_selection import SelectKBest, \
                                      SelectFromModel, \
                                      RFE, SelectPercentile, \
                                      SelectFdr, SelectFpr
from sklearn.feature_selection import f_regression, f_classif
from sklearn.neighbors import KNeighborsClassifier

### Pipeline 2: SKB, PCA, SFM, LR

In [6]:
pipe_for_gs = Pipeline([
    ('skb', SelectKBest(score_func=f_classif)),
    ('pca',PCA(random_state=42)),
    ('sfm', SelectFromModel(LogisticRegression())),
    ('knc', KNeighborsClassifier())
])

In [7]:
params = {    
    'skb__k':[250,500],
    'pca__n_components':[50,100,150],
    'sfm__estimator':[LogisticRegression(C=0.1,penalty='l1',random_state=42),
                      LogisticRegression(C=1,penalty='l1',random_state=42),
                      LogisticRegression(C=10,penalty='l1',random_state=42),
                      LogisticRegression(C=1e2,penalty='l1',random_state=42),
                      LogisticRegression(C=1e3,penalty='l1',random_state=42),
                      LogisticRegression(C=1e4,penalty='l1',random_state=42)]
}

In [8]:
gspipe = GridSearchCV(pipe_for_gs,
                      param_grid=params,
                      n_jobs=-1,
                      cv=ShuffleSplit(random_state=42))

In [9]:
gspipe.fit(feature[:220], np.ravel(target[:220].values))

GridSearchCV(cv=ShuffleSplit(n_splits=10, random_state=42, test_size=0.1, train_size=None),
       error_score='raise',
       estimator=Pipeline(steps=[('skb', SelectKBest(k=10, score_func=<function f_classif at 0x7f132aec26a8>)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('sfm', SelectFromModel(estimator=LogisticRegression(C=1.0, class_weight=None, dual=Fa...owski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'skb__k': [250, 500], 'sfm__estimator': [LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, wa... solver='liblinear', tol=0.0001, verbose=0, warm_start=False)], 'pca__n_components': [50, 100, 

In [10]:
gspipe.best_score_

0.53181818181818186

In [11]:
gspipe.best_estimator_.named_steps

{'knc': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=1, n_neighbors=5, p=2,
            weights='uniform'),
 'pca': PCA(copy=True, iterated_power='auto', n_components=50, random_state=42,
   svd_solver='auto', tol=0.0, whiten=False),
 'sfm': SelectFromModel(estimator=LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
         prefit=False, threshold=None),
 'skb': SelectKBest(k=500, score_func=<function f_classif at 0x7f132aec26a8>)}

In [12]:
gspipe_results = pd.DataFrame(gspipe.cv_results_)
gspipe_results.sort_values('rank_test_score', ascending=True).T

,1,5,7,9,11,3,21,28,17,2,...,8,25,15,16,12,22,24,20,18,0
mean_fit_time,0.04043,0.03818,0.04053,0.03792,0.0383,0.04076,0.04258,0.031,0.04319,0.02984,...,0.1554,0.04034,0.0418,0.03176,0.02889,0.02873,0.0277,0.02824,0.03162,0.02492
mean_score_time,0.001329,0.001488,0.001577,0.001448,0.001349,0.001399,0.002864,0.001728,0.00178,0.001486,...,0.001401,0.001902,0.001747,0.001523,0.001242,0.001821,0.001507,0.001662,0.001538,0.001181
mean_test_score,0.5318,0.5273,0.5273,0.5273,0.5273,0.5227,0.5227,0.5182,0.5136,0.5136,...,0.4909,0.4864,0.4864,0.4818,0.4818,0.4818,0.4818,0.4773,0.4773,0.4773
mean_train_score,0.7141,0.7045,0.7056,0.7056,0.7056,0.7096,0.7051,0.7111,0.7061,0.7111,...,0.7116,0.7126,0.7091,0.7177,0.7237,0.7101,0.7232,0.7106,0.7172,0.7116
param_pca__n_components,50,50,50,50,50,50,100,150,100,50,...,50,150,100,100,100,100,150,100,100,50
param_sfm__estimator,"LogisticRegression(C=0.1, class_weight=None, d...","LogisticRegression(C=10, class_weight=None, du...","LogisticRegression(C=100.0, class_weight=None,...","LogisticRegression(C=1000.0, class_weight=None...","LogisticRegression(C=10000.0, class_weight=Non...","LogisticRegression(C=1, class_weight=None, dua...","LogisticRegression(C=1000.0, class_weight=None...","LogisticRegression(C=10, class_weight=None, du...","LogisticRegression(C=10, class_weight=None, du...","LogisticRegression(C=1, class_weight=None, dua...",...,"LogisticRegression(C=1000.0, class_weight=None...","LogisticRegression(C=0.1, class_weight=None, d...","LogisticRegression(C=1, class_weight=None, dua...","LogisticRegression(C=10, class_weight=None, du...","LogisticRegression(C=0.1, class_weight=None, d...","LogisticRegression(C=10000.0, class_weight=Non...","LogisticRegression(C=0.1, class_weight=None, d...","LogisticRegression(C=1000.0, class_weight=None...","LogisticRegression(C=100.0, class_weight=None,...","LogisticRegression(C=0.1, class_weight=None, d..."
param_skb__k,500,500,500,500,500,500,500,250,500,250,...,250,500,500,250,250,250,250,250,250,250
params,"{'pca__n_components': 50, 'sfm__estimator': Lo...","{'pca__n_components': 50, 'sfm__estimator': Lo...","{'pca__n_components': 50, 'sfm__estimator': Lo...","{'pca__n_components': 50, 'sfm__estimator': Lo...","{'pca__n_components': 50, 'sfm__estimator': Lo...","{'pca__n_components': 50, 'sfm__estimator': Lo...","{'pca__n_components': 100, 'sfm__estimator': L...","{'pca__n_components': 150, 'sfm__estimator': L...","{'pca__n_components': 100, 'sfm__estimator': L...","{'pca__n_components': 50, 'sfm__estimator': Lo...",...,"{'pca__n_components': 50, 'sfm__estimator': Lo...","{'pca__n_components': 150, 'sfm__estimator': L...","{'pca__n_components': 100, 'sfm__estimator': L...","{'pca__n_components': 100, 'sfm__estimator': L...","{'pca__n_components': 100, 'sfm__estimator': L...","{'pca__n_components': 100, 'sfm__estimator': L...","{'pca__n_components': 150, 'sfm__estimator': L...","{'pca__n_components': 100, 'sfm__estimator': L...","{'pca__n_components': 100, 'sfm__estimator': L...","{'pca__n_components': 50, 'sfm__estimator': Lo..."
rank_test_score,1,2,2,2,2,6,6,8,9,9,...,23,28,28,30,30,30,30,34,34,34
split0_test_score,0.5455,0.6364,0.6364,0.6364,0.6364,0.6364,0.6364,0.7727,0.5909,0.6818,...,0.6818,0.5455,0.6364,0.5909,0.5909,0.6364,0.5909,0.6364,0.5909,0.5909


In [13]:
gspipe_results['mean_fit_time'].sum()

1.6438697099685669

In [14]:
gspipe_results.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 33 columns):
mean_fit_time              36 non-null float64
mean_score_time            36 non-null float64
mean_test_score            36 non-null float64
mean_train_score           36 non-null float64
param_pca__n_components    36 non-null object
param_sfm__estimator       36 non-null object
param_skb__k               36 non-null object
params                     36 non-null object
rank_test_score            36 non-null int32
split0_test_score          36 non-null float64
split0_train_score         36 non-null float64
split1_test_score          36 non-null float64
split1_train_score         36 non-null float64
split2_test_score          36 non-null float64
split2_train_score         36 non-null float64
split3_test_score          36 non-null float64
split3_train_score         36 non-null float64
split4_test_score          36 non-null float64
split4_train_score         36 non-null float64
split5_tes

In [15]:
print("Fit time .1% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score .1% data: ",gspipe.best_score_)
### Pipeline2 for .1% (220 rows)
picked_pipeline_220row = \
        pd.DataFrame.to_pickle(gspipe_results,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline2_knc_pt1_percent')

Fit time .1% data:  1.64386970997
Best Score .1% data:  0.531818181818


In [16]:
### Pipeline2 for .5% (1100 rows)

gspipe.fit(feature[:1100], np.ravel(target[:1100].values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time .5% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score .5% data: ",gspipe.best_score_)
gspipe_results_1100 = pd.DataFrame(gspipe.cv_results_)
picked_Pipeline2_1100row = \
        pd.DataFrame.to_pickle(gspipe_results_1100,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline2_knc_pt5_percent')

Fit time .5% data:  5.27950110435
Best Score .5% data:  0.537272727273


In [17]:
### Pipeline2 for 1% (2200 rows)

gspipe.fit(feature[:2200], np.ravel(target[:2200].values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time 1% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score 1% data: ",gspipe.best_score_)
gspipe_results_2200 = pd.DataFrame(gspipe.cv_results_)
picked_Pipeline2_2200row = \
        pd.DataFrame.to_pickle(gspipe_results_2200,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline2_knc_1_percent')

Fit time 1% data:  9.47058827877
Best Score 1% data:  0.527727272727


In [18]:
### Bench Marking for 5% (11000 rows)

gspipe.fit(feature, np.ravel(target.values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time 5% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score 5% data:",gspipe.best_score_)
gspipe_results_11000 = pd.DataFrame(gspipe.cv_results_)
picked_GS_BenchMarkig_11000row = \
        pd.DataFrame.to_pickle(gspipe_results_11000,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline2_knc_5_percent')

Fit time 5% data:  44.2724800825
Best Score 5% data: 0.526555246054
